In [1]:
from google.colab import drive
import os

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/Colab Notebooks/GrowthBotics/wood defects files/Wood Detector/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import glob
import random
import shutil

# Resize images to 800x600

In [0]:
for loc in ['defect - peeled bark', 'non-defect']:
    print(f'Processing {loc}...')
    for file in glob.glob(f'../wood defects 2/wood defects 2/{loc}/*.jpg'):
        image = Image.open(file)
        new_img = image.resize((800, 600), Image.ANTIALIAS)
        new_img.save(f'resized_images/{loc}/{os.path.basename(file)}')
print(f'# Images in resized_images/defect - peeled bark/ ---> {len(os.listdir("resized_images/defect - peeled bark/"))}')
print(f'# Images in resized_images/non-defect/ ---> {len(os.listdir("resized_images/non-defect/"))}')

# Script - Image Folder structure creation

In [0]:
%%time

files = [x.split('\\')[-1].split('.xml')[0] for x in glob.glob(f'annotations\*.xml')]
file_count = len(files)
train_idx = random.sample(range(file_count), round(0.9*file_count))

for i in range(file_count):
    for file_loc in glob.glob(f'annotations\{files[i]}.*'):
        file = file_loc.split('\\')[-1]
        if i in train_idx:
            shutil.copyfile(os.path.join('annotations', file), os.path.join('annotations\\train', file))
        else:
            shutil.copyfile(os.path.join('annotations', file), os.path.join('annotations\\test', file))
            
print(f'# XML Files: {file_count}')
print(f'# Files in Training Set --> {len(os.listdir("annotations/train"))}')
print(f'# Files in Test Set --> {len(os.listdir("annotations/test"))}')

# XML Files: 228
# Files in Training Set --> 410
# Files in Test Set --> 46
Wall time: 4.35 s


# Script - XML to CSV

In [3]:
# https://github.com/datitran/raccoon_dataset/blob/master/xml_to_csv.py
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    for file_dir in ['train', 'test']:
        print(f'Processing {file_dir}...')
        image_path = os.path.join('annotations', file_dir)
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv(f'data/{file_dir}_labels.csv', index=None)
        print('Successfully converted xml to csv.')


main()

Processing train...
Successfully converted xml to csv.
Processing test...
Successfully converted xml to csv.
